In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

In [ ]:
BUCKET=f"your-bucket-name-{PROJECT_ID}-unique"

In [ ]:
TAG="sdxl-serve:latest"
REPO="vertexai"
SERVE_DOCKER_URI=f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{REPO}/{TAG}"

In [ ]:
!gcloud artifacts repositories create $REPO --repository-format=docker --location=$REGION --description="Docker repository"

In [ ]:
# make sure you executed this in a terminal
# !gcloud auth configure-docker europe-west4-docker.pkg.dev

In [ ]:
!docker build -f Dockerfile . -t $TAG

In [ ]:
!docker tag $TAG $SERVE_DOCKER_URI

In [ ]:
!docker push $SERVE_DOCKER_URI

# Deploy

In [ ]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=f"gs://{BUCKET}")

In [ ]:
import base64
from io import BytesIO

from google.cloud import aiplatform
from PIL import Image


# Convert base64 encoded string to an image.
def base64_to_image(image_str):
    image = Image.open(BytesIO(base64.b64decode(image_str)))
    return image

In [ ]:
model_name = "stable-diffusion-xl"
model_id="stabilityai/stable-diffusion-xl-base-1.0"
refiner_model_id="stabilityai/stable-diffusion-xl-refiner-1.0"
task="text-to-image"
endpoint = aiplatform.Endpoint.create(display_name=f"{model_name}-{task}-endpoint")
serving_env = {
    "MODEL_ID": model_id,
    "REFINER_MODEL_ID": refiner_model_id,
    "TASK": task,
}

In [ ]:
endpoint = aiplatform.Endpoint.list(filter=f"display_name=stable-diffusion-xl-text-to-image-endpoint")[0]

In [ ]:
# endpoint.undeploy_all()

In [ ]:
model = aiplatform.Model.list(filter=f"display_name=stable-diffusion-xl")[0]

In [ ]:
# model.delete()

In [ ]:
model_name = "stable-diffusion-xl"
model_id="stabilityai/stable-diffusion-xl-base-1.0"
refiner_model_id="stabilityai/stable-diffusion-xl-refiner-1.0"
task="text-to-image"
serving_env = {
        "MODEL_ID": model_id,
        "REFINER_MODEL_ID": refiner_model_id,
        "TASK": task,
    }
model = aiplatform.Model.upload(
        display_name=model_name,
        serving_container_image_uri=SERVE_DOCKER_URI,
        serving_container_ports=[7080],
        serving_container_predict_route="/predictions/diffusers_serving",
        serving_container_health_route="/ping",
        serving_container_environment_variables=serving_env,
    )

In [ ]:
%%time

model.deploy(
    endpoint=endpoint,
    machine_type="g2-standard-4", # "a2-highgpu-1g" # "g2-standard-4" # "a2-ultragpu-1g"  
    accelerator_type="NVIDIA_L4", # "NVIDIA_TESLA_A100" # "NVIDIA_L4" # "NVIDIA_A100_80GB" 
    accelerator_count=1,
    deploy_request_timeout=1800,
)

In [ ]:
%%time

instances = [
    {"prompt": "An astronaut riding a horse on Mars, 4k photo", "negative_prompt":"", "height": 768, "width": 768, "demoising_end":0.8, "seed": 613},
]
response = endpoint.predict(instances=instances)
images = [base64_to_image(image) for image in response.predictions]
images[0]